<a href="https://colab.research.google.com/github/Igorps023/SegurancaPublicaSSP/blob/main/02refined_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=eaf32ff524ec5c218106d62983ef1e2f5e7a1d991a66182c0a0b0ee4501bf3d7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
#!/usr/bin/env python
# coding: utf-8

In [5]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg, col

In [6]:
### FREQ FUNCTION
def Freq(pTabela,pColuna):

    qtd_total=pTabela.count()

    pTabela.registerTempTable("tab_input")
    frq = spark.sql(
            """
                select
                    {col},
                    count(*) as qtd_absoluto,
                    round(100*(count(*) / {tot}),2) as qtd_percentual
                from
                    tab_input
                group by
                    {col}
                order by
                    {col}
            """.format(col=pColuna, tot=qtd_total))

    qtd=frq.count()
    print('Quantidade de dominios',qtd)
    if qtd > 500:
        frq.show(1000,truncate=False)
        return "Dominio muito granular"

    else:
        frq.show(qtd,truncate=False)
        print("volumetria total:",qtd_total)
        return 'Freq da coluna ' + pColuna;

In [7]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

In [8]:
dthproc

'20240109195604'

In [9]:
### CHANGE FOR EACH FILE
spark = SparkSession.builder.appName('t_ssp_montly_file').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
### CHANGE FOR EACH FILE
raw ='t_ssp_montly_file'

In [11]:
bucket_control =    "bkt-ssp/Files/5_Control"
bucket_ingestion =  "bkt-ssp/Files/1_IngestionZone"
bucket_raw =        "bkt-ssp/Files/2_RawZone"
bucket_trusted =    "bkt-ssp/Files/3_TrustedZone"
bucket_refined =    "bkt-ssp/Files/4_RefinedZone"

In [12]:
### CHANGE FOR EACH FILE
output_trusted = 'trusted_ssp_montly_file'

In [13]:
full_path_raw = 's3://{bkt}/{file}'.format(bkt=bucket_raw,file=raw)
print(full_path_raw)

s3://bkt-ssp/Files/2_RawZone/t_ssp_montly_file


In [ ]:
path_lake = os.path.join(full_path_raw)
raw_00=sqlContext.read.format("parquet").load(path_lake)
raw_00.registerTempTable("raw_00")

In [ ]:
raw_00.printSchema()

In [ ]:
lake = spark.sql(
    """
        select
            -- padrao para todas as cargas no datalake--

            --ref,
            --ref_partition,

            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,

            -- campos do arquivo --

            cast(track_id as string) as track_id,
            cast(user_id as string) as user_id,
            cast(playcount as bigint) as playcount
        from
            raw_00
    """.format(pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9711301

In [ ]:
### CHANGE FOR EACH FILE
path_trusted = os.path.join('s3://',bucket_trusted,output_trusted)
print(path_trusted)

lake.coalesce(1).write.parquet(path_trusted,mode='overwrite')
#partitionBy("genre","ts_proc_partition"). \
#parquet(path_trusted,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/TrustedZone/trusted_user_listening_history

In [ ]:
varDataFile = output_trusted

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
controle = spark.sql(
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,

                --ref,
                --ref_partition,

                ts_proc,
                ts_proc_partition,

                count(*) as qtd_registros
            from
                lake as a
            group by
                1,2,3,4
            order by
                1,2,3,4
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
path_control = os.path.join('s3://',bucket_control,'tb_0002_controle_procesamento_trusted')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-musicstream-bi/Files/Control/tb_0002_controle_procesamento_trusted

In [ ]:
# End